# Load Packages

In [1]:
using Dates,DelimitedFiles, JuMP, Ipopt, HDF5

include("jlFiles/printmat.jl")
include("jlFiles/printTable.jl")

printTable2

In [2]:
using Plots

#pyplot(size=(600,400))           #pyplot or gr
gr(size=(480,320))
default(fmt = :svg)    

# Loading Data


from a hdf5 file (a common data format, used by many languages). 

See https://support.hdfgroup.org/products/java/hdfview/
for a program that lets you look at the contents of a hdf5 file. (It is not needed here.)

In [3]:
fh = h5open("Data/Fin1PerfEvalEmp.h5","r")   #open for reading
  (ymd,Rb,IndNames,RFunds,FundNames) = read(fh,"Data/ymd","Data/Rb",
                                               "Data/IndNames","Data/RFunds",
                                               "Data/FundNames")
close(fh)
dN = Date.(ymd[:,1],ymd[:,2],ymd[:,3])       #2001,12,31 to julia date

println("\nThe funds: ")
for i = 1:length(FundNames)
    println(FundNames[i])
end

println("\nIndices: ")
for i = 1:length(IndNames)
    println(IndNames[i])
end


The funds: 
Putnam Asset Allocation: Growth A
Vanguard Wellington

Indices: 
Equity: US, LargeCap, Value
Equity: US, LargeCap, Growth
Equity: US, MidCap
Equity: US, SmallCap
Equity: Int. (ex US), Developed
Equity: Emerging Markets
Fixed Income: US, Bills
Fixed Income: US, Gov. Bonds
Fixed Income: US, Corp. Bonds
Fixed Income: Non-US, Bonds


# A Function for Style Analysis


Notice:  
(a)  The regression is $Y = b_1 X_1 + ...+ b_K X_K + u$,
               where $0 <= b_j$ and $\sum_{i=1}^{K}b_i = 1$. 
               
(b) Write the sum of squared residuals of the regression as   

$(Y-Xb)'(Y-Xb) = Y'Y - 2Y'Xb + b'X'Xb$. 

Only the two last terms matter for the choice of $b$. 
                             
(c) To minimize $- 2Y'Xb + b'X'Xb$, we use the JuMP package. 

In [4]:
function StyleAnalysisPs(Y,X)

    (T,K) = (size(X,1),size(X,2))
    Q = X'X/T                           #calculate Q and c once. 1/T may help precision       
    c = Y'X/T    

    b_ls = X\Y                          #LS estimate of weights, no restrictions
 
    model = Model(with_optimizer(Ipopt.Optimizer, print_level = 1))
    @variable(model,b[i=1:K] >= 0.0)       #no short sales
    @objective(model,Min,-2*c*b+b'*Q*b)    #sum of squared errors
    @constraint(model,sum(b) == 1.0)       #sums to 1
    optimize!(model)
    if has_values(model)
        b_sa = value.(b)
    else
        b_sa = NaN
    end
 
    return b_sa,b_ls

end

StyleAnalysisPs (generic function with 1 method)

The next cell makes a "style analysis regression" based on the entire sample. The dependent variable is the first mutual fund in RFunds (see data loading) and all indices (again, see data loading) are used as regressors.

In [5]:
(b,b_ls) = StyleAnalysisPs(RFunds[:,1],Rb)

println("OLS and style analysis coeffs")
colNames = ["OLS" "Restricted LS"]
xut = [b_ls b;sum([b_ls b],dims=1)]
printTable(xut,colNames,[IndNames;"Sum"],width=15)

vvSort = sortperm(b,rev=true)    #indices so that b[vvSort] is sorted in descending order
printlnPs("\nthe 3 most important benchmarks are: ")
printmat(IndNames[vvSort[1:3]])


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

OLS and style analysis coeffs
                                           OLS  Restricted LS
Equity: US, LargeCap, Value              0.129          0.130
Equity: US, LargeCap, Growth             0.098          0.098
Equity: US, MidCap                       0.079          0.078
Equity: US, SmallCap                     0.061          0.062
Equity: Int. (ex US), Developed          0.413          0.413
Equity: Emerging Markets                 0.047          0.049
Fixed Income: US, Bills                  0.091          0.063
Fixed Income: US, Gov. Bonds            -0.065          0.002
Fixed Income: US, Corp. 

# Redo the Style Analysis on a Moving Data Window 

of size `WinSize` (see below). Then plot to see how the coefficients change over time. 

In [6]:
(T,K)   = size(Rb)
WinSize = 104

b = fill(NaN,(T,K))
for t = (WinSize+1):T
    local vv
    vv     = (t-WinSize):t
    b[t,:] = StyleAnalysisPs(RFunds[vv,1],Rb[vv,:])[1]
end

In [7]:
xTicksLoc = Dates.value.([Date(2000);Date(2005);Date(2010);Date(2015)])
xTicksLab = ["2000";"2005";"2010";"2015"]              

p1 = plot(dN,b[:,vvSort[1:3]],color=[:blue :red :green],label=IndNames[vvSort[1:3]],
          xticks=(xTicksLoc,xTicksLab),size=(600,400))
title!("Loadings on 3 key benchmarks")
annotate!(Dates.value(Date(2012)),0.03,text("(moving data window of $WinSize months)",9))